In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
%%bash
pip install tiktoken
pip install deepspeed
cd LLaMA-Factory
pip install -r requirements.txt

### 目前我們支援alpaca或sharegpt格式的資料集，alpaca格式的資料集應遵循以下格式：
```
[
  {
    "instruction": "user instruction (required)",
    "input": "user input (optional)",
    "output": "model response (required)",
    "system": "system prompt (optional)",
    "history": [
      ["user instruction in the first round (optional)", "model response in the first round (optional)"],
      ["user instruction in the second round (optional)", "model response in the second round (optional)"]
    ]
  }
]
```

### 對於上面的資料集，columnsindataset_info.json應該是：
```
"dataset_name": {
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
    "system": "system",
    "history": "history"
  }
}
```
>該query列將與該列連接prompt並用作用戶提示，則用戶提示將為prompt\nquery。該response列代表模型響應。
該system列將用作系統提示符。該history列是一個列表，由表示歷史記錄中的提示-響應對的字串元組組成。請注意，歷史記錄中的反應也將用於訓練。
對於預訓練資料集，僅prompt使用該列進行訓練。

### 對於偏好資料集，該response列應該是長度為 2 的字串列表，首選答案首先出現，例如：
```
{
  "instruction": "user instruction",
  "input": "user input",
  "output": [
    "chosen answer",
    "rejected answer"
  ]
}
```

### ASK GPT to write a pytohn code   xlsx2jsonl.py
```
Write a python file for me. using openpyxl. input file name is MedQA2019.xlsx
Step1: The input file is .xlsx. Exact the column A and column D in the sheet named "DrugQA" .
Step2: Put each value in column A into each "instruction" of each "conversation". Put each value in column D into each "output" of each "conversation".
Step3: The output file is MedQA2019.json. It looks like:
[
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  }
]

Step4: All "system" value changes to "You are a professional, highly experienced doctor professor. You always provide accurate, comprehensive, and detailed answers based on the patients' questions."
```

In [ ]:
pip install openpyxl

In [ ]:
## GPT CODE
import openpyxl
import json

# Step 1: Extract columns A and D from the sheet named "DrugQA"
def extract_data_from_excel(file_name):
    workbook = openpyxl.load_workbook(file_name)
    sheet = workbook['DrugQA']
    
    instructions = [cell.value for cell in sheet['A']]
    outputs = [cell.value for cell in sheet['D']]
    
    workbook.close()
    
    return instructions, outputs

# Step 2: Create conversation objects
def create_conversations(instructions, outputs):
    conversations = []
    for instruction, output in zip(instructions, outputs):
        conversation = {
            "instruction": instruction,
            "input": "",
            "output": output,
            "system": ""  # Will be updated in Step 4
        }
        conversations.append(conversation)
    return conversations

# Step 3: Write conversations to JSON file
def write_to_json(conversations, output_file):
    with open(output_file, 'w') as f:
        json.dump(conversations, f, indent=2)

# Step 4: Update system values
def update_system_values(conversations):
    for conversation in conversations:
        conversation["system"] = "You are a professional, highly experienced doctor professor. You always provide accurate, comprehensive, and detailed answers based on the patients' questions."

# Main function to orchestrate the process
def main(input_file, output_file):
    instructions, outputs = extract_data_from_excel(input_file)
    conversations = create_conversations(instructions, outputs)
    update_system_values(conversations)
    write_to_json(conversations, output_file)

if __name__ == "__main__":
    input_file = "MedQA2019.xlsx"
    output_file = "MedQA2019.json"
    main(input_file, output_file)


### ASK GPT AGAIGN
- ask GPT
```
my MedQA2019.json file looks like:
[
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  },
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  }
]
Step1, read the MedQA2019.json file.
Step2, count the amount of the "instruction" elements.
Step3, randomly split all "instruction" elements by 7:3. Targeted structure is same as the input.
Step4, save the 7/10 part as MedQA2019_train.json. save the 3/10 part as MedQA2019_test.json
```

In [ ]:
import json
import random

# Step 1: Read MedQA2019.json file
def read_json(file_name):
    with open(file_name, 'r') as f:
        data = json.load(f)
    return data

# Step 2: Count the amount of "instruction" elements
def count_instructions(data):
    return len(data)

# Step 3: Randomly split "instruction" elements by 7:3
def split_data(data):
    random.shuffle(data)
    split_index = int(len(data) * 0.7)
    train_data = data[:split_index]
    test_data = data[split_index:]
    return train_data, test_data

# Step 4: Save the split data into train.json and test.json
def save_to_json(train_data, test_data):
    with open('MedQA2019_train.json', 'w') as train_file:
        json.dump(train_data, train_file, indent=2)
    with open('MedQA2019_test.json', 'w') as test_file:
        json.dump(test_data, test_file, indent=2)

# Main function to orchestrate the process
def main(input_file):
    data = read_json(input_file)
    total_instructions = count_instructions(data)
    print("Total number of instructions:", total_instructions)
    train_data, test_data = split_data(data)
    print("Number of instructions in MedQA2019_train.json:", len(train_data))
    print("Number of instructions in MedQA2019_test.json:", len(test_data))
    save_to_json(train_data, test_data)

if __name__ == "__main__":
    input_file = "MedQA2019.json"
    main(input_file)

### dataset_info.json (LLaMA-Factory/data/dataset_info.json)
Regarding the above dataset, the columns in dataset_info.json should be:
```
"dataset_name": {
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
    "system": "system",
    "history": "history"
  }
}
```


```
  "medical": {
    "file_name": "MedQA2019_train.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output",
      "system": "system",        
    }
  },   
```

In [ ]:
!sha1sum LLaMA-Factory/data/MedQA2019_train.json

In [ ]:
## DTAT2

### ASK GPT to write a pytohn code   xlsx2jsonl.py
```
Write a python file for me. using openpyxl

1. 我有一個文件input.josn，每一行的格式為JSON格式，如下所示：
{"name": "西班牙鮪魚沙拉", "dish": "鮪魚沙拉", "description": "", "recipeIngredient": ["超市罐頭半盒鮪魚(泉水)", "2大片生菜」, "5個小番茄", "半根黃瓜", "半紅柿椒", "半個紫洋蔥", "1個七成熟水煮蛋", "遠程紅酒醋", "遠程胡椒" "， "橄欖油"]，"食譜說明"：["雞蛋進水煮，七成熟撈出（依個人喜好），同時備其他菜"，"生菜撕片，女果開半，黃瓜滾刀，紅柿椒切絲，紫洋蔥切絲，雞蛋四均分"，"金槍魚去水"，"撒黑胡椒，紅酒醋和少許橄欖油"，"拌勻，拍照，開動"]，"作者" : "author_67696 ", "keywords": ["西班牙鮪魚沙拉的做法", "西班牙鮪魚沙拉的家常做法", "西班牙鮪魚沙拉的詳細做法", "西班牙鮪魚沙拉怎麼做", "西班牙鮪魚沙拉的最正宗做法" , "沙拉"]}
2. 我需要產生一個以下為範本格式的json文件：
[
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  }
]
3. instruction 取自將原有JSON格式中 keywords 中的任何一個
4. input 為空白
5. output 將原有JSON格式中的 recipeIngredient 和 recipeInstructions 组合起来，描述成：
您需要準備以下食材：
recipeIngredient
按以下方法製作：
recipeInstructions
6. 如果原有JSON格式中 keywords 有多少筆，則產生多條JSON記錄
7. 使用openpyxl库。生成好Python脚本给我。
8. 輸出的檔案名稱為 output.josn
```

### ASK GPT to write a pytohn code   xlsx2jsonl.py
```
Write a python file for me. using openpyxl

1. 我有一個文件 input.josn，每一行的格式為JSON格式，如下所示：
{"name": "西班牙鮪魚沙拉", "dish": "鮪魚沙拉", "description": "", "recipeIngredient": ["超市罐頭半盒鮪魚(泉水)", "2大片生菜」, "5個小番茄", "半根黃瓜", "半紅柿椒", "半個紫洋蔥", "1個七成熟水煮蛋", "遠程紅酒醋", "遠程胡椒" "， "橄欖油"]，"食譜說明"：["雞蛋進水煮，七成熟撈出（依個人喜好），同時備其他菜"，"生菜撕片，女果開半，黃瓜滾刀，紅柿椒切絲，紫洋蔥切絲，雞蛋四均分"，"金槍魚去水"，"撒黑胡椒，紅酒醋和少許橄欖油"，"拌勻，拍照，開動"]，"作者" : "author_67696 ", "keywords": ["西班牙鮪魚沙拉的做法", "西班牙鮪魚沙拉的家常做法", "西班牙鮪魚沙拉的詳細做法", "西班牙鮪魚沙拉怎麼做", "西班牙鮪魚沙拉的最正宗做法" , "沙拉"]}
2. 我需要產生一個以下為範本格式的json文件：
[
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  }
]
3. instruction 取自將原有JSON格式中 keywords 中的任何一個
4. input 為空白
5. output 將原有JSON格式中的 recipeIngredient 和 recipeInstructions 组合起来，描述成：
您需要準備以下食材：
recipeIngredient
按以下方法製作：
recipeInstructions
6. 如果原有JSON格式中 keywords 有多少筆，則產生多條JSON記錄
7. 使用openpyxl库。生成好Python脚本给我。
8. 輸出的檔案名稱為 output.josn
```

### ASK GPT AGAIGN
- ask GPT
```
Write a python file for me. using openpyxl. input file name is MedQA2019.jsonl

my MedQA2019.json file looks like:
{"name": "西班牙金枪鱼沙拉", "dish": "金枪鱼沙拉", "description": "", "recipeIngredient": ["超市罐头装半盒金枪鱼(in spring water)", "2大片生菜", "5个圣女果", "半根黄瓜", "半个红柿椒", "半个紫洋葱", "1个七成熟水煮蛋", "适量红酒醋", "适量胡椒", "适量橄榄油"], "recipeInstructions": ["鸡蛋进水煮，七成熟捞出（依个人喜好），同时备其他菜", "生菜撕片，圣女果开半，黄瓜滚刀，红柿椒切丝，紫洋葱切丝，鸡蛋四均分", "金枪鱼去水", "撒黑胡椒，红酒醋和少许橄榄油", "拌匀，拍照，开动"], "author": "author_67696", "keywords": ["西班牙金枪鱼沙拉的做法", "西班牙金枪鱼沙拉的家常做法", "西班牙金枪鱼沙拉的详细做法", "西班牙金枪鱼沙拉怎么做", "西班牙金枪鱼沙拉的最正宗做法", "沙拉"]}
{"name": "红烧滩羊肉", "dish": "Unknown", "description": "每到桂花香满城后，就可以吃羊肉温补身体了", "recipeIngredient": ["1kg羊肉", "5片姜", "3瓣蒜", "适量花椒", "3勺老抽", "3片香叶", "2个八角", "1个干辣椒", "1块桂皮", "2勺料酒", "适量盐", "3根香菜", "2个小洋葱", "适量乱泉水"], "recipeInstructions": ["滩羊肉在姜水里焯3分钟，姜水里加点料酒去腥", "再将羊肉反过来焯水2分钟", "把羊肉切2/3手掌大小，羊肉煮熟后会缩水，所以可以稍微大一点的", "热油里放入花椒、桂皮、香叶、生姜爆炒30秒，放入切好的羊肉翻炒3分钟左右，把羊肉里面的油煸炒出来，再加入老抽上色，把所有羊肉都上色后加入乱泉水，以漠过羊肉上面为准", "锅里水烧开后，换成砂锅中火慢炖40分钟", "分次吃", "超级软糯", "加点萝卜进去炖起来，解油腻"], "author": "author_25682", "keywords": ["红烧滩羊肉的做法", "红烧滩羊肉的家常做法", "红烧滩羊肉的详细做法", "红烧滩羊肉怎么做", "红烧滩羊肉的最正宗做法"]}

Step1, select one keyword of keywords as instruction

Step2, keep input empty

Step3, output is like string below
您需要準備以下食材：
recipeIngredient
按以下方法製作：
recipeInstructions

Step4: "system" value is "你是個專業的廚師，你會做很多菜。用戶報上自己所需的菜名後，你可以把做飯所需的原料，以及做飯的方法告訴用戶"

Step5: The output file is out.json. It looks like:
[
  {
    "instruction": "xxx",
    "input": "xxx",
    "output": "xxx",
    "system": "xxx",
  }
]
```

In [ ]:
import json
import openpyxl

# Step 1: Select one keyword of keywords as instruction
def select_instruction(keywords):
    return keywords[0]

# Step 2: Keep input empty
def keep_input_empty():
    return ""

# Step 3: Generate output string
def generate_output(recipe_ingredients, recipe_instructions):
    output = "您需要準備以下食材：\n"
    output += "\n".join(recipe_ingredients) + "\n"
    output += "按以下方法製作：\n"
    output += "\n".join(recipe_instructions)
    return output

# Step 4: Define system value
system_value = "你是個專業的廚師，你會做很多菜。用戶報上自己所需的菜名後，你可以把做飯所需的原料，以及做飯的方法告訴用戶"

# Step 5: Process JSON file and generate output
output_data = []
with open("XiaChuFang_Recipe_Corpus/recipe_corpus_full.json", "r", encoding="utf-8") as file:
    for line in file:
        recipe_data = json.loads(line)
        instruction = select_instruction(recipe_data["keywords"])
        input_data = keep_input_empty()
        output = generate_output(recipe_data["recipeIngredient"], recipe_data["recipeInstructions"])
        output_data.append({
            "instruction": instruction,
            "input": input_data,
            "output": output,
            "system": system_value
        })

# Write output to out.json
with open("out.json", "w", encoding="utf-8") as out_file:
    json.dump(output_data, out_file, ensure_ascii=False, indent=2)


In [ ]:
## OPENCC

In [ ]:
!pip install opencc
!pip install opencc-python-reimplemented

In [ ]:
import json
from datasets import load_dataset
import opencc #繁体简体互转
#op_cc=opencc.OpenCC('s2t')
op_cc=opencc.OpenCC('s2twp')

In [ ]:
# 所有資料內容
import pandas as pd
data = pd. read_json ( 'out.json' )
df = pd.DataFrame(data) # 轉成 DataFrame


# 提取所需欄位並建立新的字典列表
limit=10000
extracted_data = []
for index, row in df.iterrows():
    instruction=row['instruction']
    output=row['output']
    system=row['system']
    extracted_example = {
        "instruction": op_cc.convert(instruction),
        "input": "",
        "output": op_cc.convert(output),
        "system": op_cc.convert(system)
    }
    extracted_data.append(extracted_example)
    if len(extracted_data) == limit:
        break

# 指定 JSON 文件名稱
json_filename = "data.json"

# 寫入 JSON 文件
with open(json_filename, "w") as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"數據已提取並保存為 {json_filename}")
    




In [ ]:
# 所有資料內容
import pandas as pd
df = pd. read_json ( 'data.json' )
df

In [3]:
import json

# Step 1: Function to select one keyword from keywords as instruction
def select_instruction(keywords):
    if keywords:
        return keywords[0]  # Select the first keyword
    else:
        return ""

# Step 3: Generate output content function
def generate_output(recipe_ingredients, recipe_instructions):
    return f"您需要準備以下食材：\n{recipe_ingredients}\n按以下方法製作：\n{recipe_instructions}"

# Step 4: System value
system_value = "你是個專業的廚師，你會做很多菜。用戶報上自己所需的菜名後，你可以把做飯所需的原料，以及做飯的方法告訴用戶"

# Step 5: Output file path
output_file = "out.json"

# Read input file and process each line
with open("MedQA2019.json", "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

output_data = []

# Process each entry in the input data
for entry in data:
    instruction = select_instruction(entry.get("keywords", []))
    recipe_ingredients = ", ".join(entry.get("recipeIngredient", []))
    recipe_instructions = ", ".join(entry.get("recipeInstructions", []))

    # Step 3: Generate output content
    output_content = generate_output(recipe_ingredients, recipe_instructions)

    output_data.append({
        "instruction": instruction,
        "input": "",
        "output": output_content,
        "system": system_value
    })

# Write the output data to a JSON file
with open(output_file, "w", encoding="utf-8") as outfile:
    json.dump(output_data, outfile, ensure_ascii=False, indent=2)


JSONDecodeError: Expecting value: line 1 column 2 (char 1)